In [92]:
from __future__ import division
import pandas as pd
import numpy as np

In [4]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# First try random labels to set a baseline. Score: 0.09.

In [24]:
image_ids = test.index + 1
random_labels = np.random.choice(range(10), size=test.shape[0], replace=True)
submission = pd.DataFrame({'ImageId': image_ids, 'Label': random_labels})
submission.to_csv("submission.csv", index=False)
submission.shape

(28000, 2)

In [17]:
pd.read_csv("sample_submission.csv").head()

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


# Now try the easiest supervised learning algo.

In [25]:
from sklearn import svm

In [39]:
X = [[0,0,0], [1,1,1], [2,2,2], [1,1,1], [2,2,2], [0,0,0]]
y = [0, 1, 2, 1, 2, 0]
clf = svm.SVC()
clf.fit(X, y)
clf.predict([2,2,2])

array([2])

In [47]:
train.head().drop('label', axis=1)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
actual_labels = train['labels'].values

In [114]:
# train it on the first 500:
size = 500
df = train[:size]
X = df.drop('label', axis=1).values
y = df['label'].values

In [115]:
clf = svm.SVC()
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [116]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X, y) 



TypeError: __init__() got an unexpected keyword argument 'decision_function_shape'

In [112]:
# test it on the next 500:
prediction_size = 200
df = train[size:size + prediction_size]
df.head()
y_predicted = clf.predict(df.drop('label', axis=1).values)

In [113]:
sum(df['label'].values == y_predicted)/prediction_size

0.12